In [3]:
#import packages
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
#download dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [5]:
# extract the training and testing dataset 
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [6]:
#split train into train and validation

# number of validation samples - 10 % of the train samples
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# number of test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    # possible values for the inputs are 0 to 255 - all elements will be between 0 and 1 
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [8]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [9]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [10]:
#create bacthes for sgd
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

In [11]:
#building the model


input_size = 784
output_size = 10
hidden_layer_size = 50
    
# define model
model = tf.keras.Sequential([
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(train_data, epochs=10, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/10
540/540 - 9s - loss: 0.4251 - accuracy: 0.8803 - val_loss: 0.2224 - val_accuracy: 0.9378 - 9s/epoch - 16ms/step
Epoch 2/10
540/540 - 4s - loss: 0.1803 - accuracy: 0.9477 - val_loss: 0.1514 - val_accuracy: 0.9553 - 4s/epoch - 8ms/step
Epoch 3/10
540/540 - 4s - loss: 0.1346 - accuracy: 0.9607 - val_loss: 0.1275 - val_accuracy: 0.9622 - 4s/epoch - 7ms/step
Epoch 4/10
540/540 - 4s - loss: 0.1096 - accuracy: 0.9671 - val_loss: 0.1092 - val_accuracy: 0.9678 - 4s/epoch - 8ms/step
Epoch 5/10
540/540 - 4s - loss: 0.0963 - accuracy: 0.9711 - val_loss: 0.0997 - val_accuracy: 0.9702 - 4s/epoch - 7ms/step
Epoch 6/10
540/540 - 4s - loss: 0.0820 - accuracy: 0.9750 - val_loss: 0.0873 - val_accuracy: 0.9732 - 4s/epoch - 7ms/step
Epoch 7/10
540/540 - 4s - loss: 0.0736 - accuracy: 0.9779 - val_loss: 0.0742 - val_accuracy: 0.9780 - 4s/epoch - 8ms/step
Epoch 8/10
540/540 - 4s - loss: 0.0640 - accuracy: 0.9808 - val_loss: 0.0829 - val_accuracy: 0.9743 - 4s/epoch - 8ms/step
Epoch 9/10
540/540 - 4s

In [14]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 1s/step - loss: 0.0929 - accuracy: 0.9726
Test loss: 0.09. Test accuracy: 97.26%
